## Deliverable 2. Create a Customer Travel Destinations Map.

In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [57]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,-17.86,90,92,5.12,overcast clouds
1,1,Sitka,US,57.0531,-135.3300,37.40,93,90,7.52,light rain
2,2,Albany,US,42.6001,-73.9662,23.00,43,0,10.00,clear sky
3,3,Barrow,US,71.2906,-156.7887,6.80,78,90,23.02,snow
4,4,Batagay-Alyta,RU,67.8006,130.4114,-39.06,75,91,0.89,overcast clouds


In [58]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [59]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
7,7,Gondanglegi,ID,-7.7479,112.0132,75.51,84,100,0.98,overcast clouds
9,9,Atuona,PF,-9.8000,-139.0333,78.12,79,2,18.39,clear sky
10,10,Buritis,BR,-15.6178,-46.4233,82.76,57,74,5.57,broken clouds
21,21,Saint-Philippe,RE,-21.3585,55.7679,80.60,74,0,5.75,clear sky
23,23,East London,ZA,-33.0153,27.9116,75.87,77,63,23.40,broken clouds


In [60]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [61]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [63]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Gondanglegi,ID,75.51,overcast clouds,-7.7479,112.0132,
9,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,
10,Buritis,BR,82.76,broken clouds,-15.6178,-46.4233,
21,Saint-Philippe,RE,80.60,clear sky,-21.3585,55.7679,
23,East London,ZA,75.87,broken clouds,-33.0153,27.9116,
24,Rikitea,PF,78.73,clear sky,-23.1203,-134.9692,
32,Rio Grande,BR,84.20,scattered clouds,-32.0350,-52.0986,
34,Bambous Virieux,MU,79.00,scattered clouds,-20.3428,57.7575,
36,Butaritari,KI,79.45,overcast clouds,3.0707,172.7902,
44,Hambantota,LK,75.20,clear sky,6.1241,81.1185,


In [64]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [75]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Gondanglegi,ID,75.51,overcast clouds,-7.7479,112.0132,Gg. Munasir Homestay
9,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,Villa Enata
10,Buritis,BR,82.76,broken clouds,-15.6178,-46.4233,Hotel Rainha do Vale
21,Saint-Philippe,RE,80.60,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
23,East London,ZA,75.87,broken clouds,-33.0153,27.9116,Tu Casa
...,...,...,...,...,...,...,...
705,Voh,NC,75.33,broken clouds,-20.9667,164.7000,Le gîte du Koniambo
713,Camopi,GF,77.09,overcast clouds,3.1655,-52.3158,
714,Cozumel,MX,84.20,broken clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort
717,Monrovia,LR,78.80,mist,6.3005,-10.7969,Sunset Inn


In [97]:
hotel_df.count()

City                   209
Country                209
Max Temp               209
Current Description    209
Lat                    209
Lng                    209
Hotel Name             209
dtype: int64

In [129]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]

In [130]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Gondanglegi,ID,75.51,overcast clouds,-7.7479,112.0132,Gg. Munasir Homestay
9,Atuona,PF,78.12,clear sky,-9.8000,-139.0333,Villa Enata
10,Buritis,BR,82.76,broken clouds,-15.6178,-46.4233,Hotel Rainha do Vale
21,Saint-Philippe,RE,80.60,clear sky,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
23,East London,ZA,75.87,broken clouds,-33.0153,27.9116,Tu Casa
...,...,...,...,...,...,...,...
704,Moroni,KM,79.66,overcast clouds,-11.7022,43.2551,Hotel les Arcades
705,Voh,NC,75.33,broken clouds,-20.9667,164.7000,Le gîte du Koniambo
714,Cozumel,MX,84.20,broken clouds,20.5083,-86.9458,Cozumel Palace® All Inclusive Resort
717,Monrovia,LR,78.80,mist,6.3005,-10.7969,Sunset Inn


In [131]:
clean_hotel_df.count()

City                   192
Country                192
Max Temp               192
Current Description    192
Lat                    192
Lng                    192
Hotel Name             192
dtype: int64

In [132]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [135]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [136]:
# 11a. Add a marker layer for each city to the map. 
#locations = clean_hotel_df[["Lat", "Lng"]]
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))